In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
p1 = Path.cwd() / 'data'

In [4]:
flights = pd.read_csv(p1 / 'flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [7]:
(flights
 .groupby('AIRLINE')[['ARR_DELAY']]
 .mean()
)

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580
F9,13.630651
HA,4.972973
MQ,6.860591
NK,18.436070


In [8]:
(flights
 .groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED']
 .sum()
)

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
                    ..
WN       3          18
         4          10
         5           7
         6          10
         7           7
Name: CANCELLED, Length: 98, dtype: int64

In [10]:
(flights
 .groupby(['AIRLINE', 'WEEKDAY'])[['CANCELLED', 'DIVERTED']]
 .agg(['sum', 'mean'])
)

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
...                   ...       ...      ...       ...
WN      3              18  0.014118        2  0.001569
        4              10  0.007911        4  0.003165
        5               7  0.005828        0  0.000000
        6              10  0.010132        3  0.003040
        7               7  0.006066        3  0.002600

[98 rows x 4 columns]

In [12]:
(flights
 .groupby(['ORG_AIR', 'DEST_AIR'])
 .agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
)

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

In [15]:
(flights
 .groupby(['ORG_AIR', 'DEST_AIR'])
 .agg(sum_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='sum'),
      mean_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='mean'),
      mean_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='mean'))
)

sum_cancelled  mean_cancelled  mean_air_time
ORG_AIR DEST_AIR                                              
ATL     ABE                   0        0.000000      96.387097
        ABQ                   0        0.000000     170.500000
        ABY                   0        0.000000      28.578947
        ACY                   0        0.000000      91.333333
        AEX                   0        0.000000      78.725000
...                         ...             ...            ...
SFO     SNA                   4        0.032787      64.059322
        STL                   0        0.000000     198.900000
        SUN                   0        0.000000      78.000000
        TUS                   0        0.000000     100.200000
        XNA                   0        0.000000     173.500000

[1130 rows x 3 columns]

In [16]:
res = (flights
 .groupby(['ORG_AIR', 'DEST_AIR'])
 .agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
)
res

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

In [23]:
res.columns = [f'{b}_{a}' for a, b in res.columns.to_flat_index()]

In [24]:
res

sum_CANCELLED  mean_CANCELLED  size_CANCELLED  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0        0.000000              31   
        ABQ                   0        0.000000              16   
        ABY                   0        0.000000              19   
        ACY                   0        0.000000               6   
        AEX                   0        0.000000              40   
...                         ...             ...             ...   
SFO     SNA                   4        0.032787             122   
        STL                   0        0.000000              20   
        SUN                   0        0.000000              10   
        TUS                   0        0.000000              20   
        XNA                   0        0.000000               2   

                  mean_AIR_TIME  var_AIR_TIME  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692  
...                         ...           ...  
SFO     SNA           64.059322     11.338331  
        STL          198.900000    101.042105  
        SUN           78.000000     25.777778  
        TUS          100.200000     35.221053  
        XNA          173.500000      0.500000  

[1130 rows x 5 columns]

In [39]:
def flatten_cols(df):
    df_copy = df.copy()
    df_copy.columns = [f'{b}_{a}' for a, b in df.columns.to_flat_index()]
    return df_copy

In [26]:
(flights
 .groupby(['ORG_AIR', 'DEST_AIR'])
 .agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
 .pipe(flatten_cols)
)

sum_CANCELLED  mean_CANCELLED  size_CANCELLED  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0        0.000000              31   
        ABQ                   0        0.000000              16   
        ABY                   0        0.000000              19   
        ACY                   0        0.000000               6   
        AEX                   0        0.000000              40   
...                         ...             ...             ...   
SFO     SNA                   4        0.032787             122   
        STL                   0        0.000000              20   
        SUN                   0        0.000000              10   
        TUS                   0        0.000000              20   
        XNA                   0        0.000000               2   

                  mean_AIR_TIME  var_AIR_TIME  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692  
...                         ...           ...  
SFO     SNA           64.059322     11.338331  
        STL          198.900000    101.042105  
        SUN           78.000000     25.777778  
        TUS          100.200000     35.221053  
        XNA          173.500000      0.500000  

[1130 rows x 5 columns]

In [40]:
air_info = (flights
            .groupby(['AIRLINE', 'WEEKDAY'])
            .agg({'DIST':['sum', 'mean'], 'ARR_DELAY':['min', 'max']})
            .astype('int')
)
air_info

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
...                  ...   ...       ...  ...
WN      3         997213   782       -38  262
        4        1024854   810       -52  284
        5         981036   816       -44  244
        6         823946   834       -41  290
        7         945679   819       -45  261

[98 rows x 4 columns]

In [41]:
air_info.columns.get_level_values(0), air_info.columns.get_level_values(1)

(Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object'),
 Index(['sum', 'mean', 'min', 'max'], dtype='object'))

In [44]:
air_info.pipe(flatten_cols).reset_index()

,AIRLINE,WEEKDAY,sum_DIST,mean_DIST,min_ARR_DELAY,max_ARR_DELAY
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
...,...,...,...,...,...,...
93,WN,3,997213,782,-38,262
94,WN,4,1024854,810,-52,284
95,WN,5,981036,816,-44,244
96,WN,6,823946,834,-41,290


In [45]:
college = pd.read_csv(p1 / 'college.csv')

In [46]:
(college
 .groupby('STABBR')['UGDS']
 .agg(['mean', 'std'])
 .round(0)
)

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0
CA,3518.0,6709.0
CO,2325.0,4670.0
CT,1874.0,2871.0
DC,2645.0,3225.0


In [47]:
def max_dev(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

In [49]:
(college
 .groupby('STABBR')['UGDS']
 .agg(max_dev)
 .round(1)
)

STABBR
AK     2.6
AL     5.8
AR     6.3
AS     NaN
AZ     9.9
CA     6.1
CO     5.0
CT     5.6
DC     2.4
DE     3.5
FL     8.4
FM     NaN
GA     5.4
GU     1.0
HI     3.8
IA     6.5
ID     4.5
IL     7.3
IN     9.1
KS     4.9
KY     5.2
LA     6.5
MA     6.1
MD     5.3
ME     4.0
MH     NaN
MI     6.7
MN     7.8
MO     7.2
MP     NaN
MS     4.0
MT     3.9
NC     4.9
ND     3.5
NE     5.0
NH     5.3
NJ     7.1
NM     4.5
NV     4.7
NY     8.2
OH    10.3
OK     5.9
OR     5.3
PA    10.1
PR     6.0
PW     NaN
RI     2.9
SC     6.0
SD     4.2
TN     6.0
TX     7.7
UT     5.1
VA     7.0
VI     NaN
VT     3.8
WA     6.6
WI     5.8
WV     7.2
WY     2.8
Name: UGDS, dtype: float64

In [57]:
def pct_bet(s, low, high):
    return round(s.between(low, high).mean() * 100, 2)

In [59]:
(college
 .groupby(['STABBR', 'RELAFFIL'])['UGDS']
 .agg(pct_bet, low=1000, high=10000)
)

STABBR  RELAFFIL
AK      0           42.86
        1            0.00
AL      0           45.83
        1           37.50
AR      0           39.71
                    ...  
WI      0           31.03
        1           44.00
WV      0           29.23
        1           37.50
WY      0           72.73
Name: UGDS, Length: 112, dtype: float64

In [60]:
def bet_n_m(n, m):
    def wrapper(ser):
        return pct_bet(ser, n, m)
    wrapper.__name__ = f'between_{n}_{m}'
    return wrapper

In [61]:
(college
 .groupby(['STABBR', 'RELAFFIL'])['UGDS']
 .agg(bet_n_m(1000, 10000))
)

STABBR  RELAFFIL
AK      0           42.86
        1            0.00
AL      0           45.83
        1           37.50
AR      0           39.71
                    ...  
WI      0           31.03
        1           44.00
WV      0           29.23
        1           37.50
WY      0           72.73
Name: UGDS, Length: 112, dtype: float64